# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [ ]:
# Install the required packages
!pip install -r requirements.txt

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
!pip install --upgrade openai

In [15]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os, dotenv
import openai

dotenv.load_dotenv(override=True)

True

In [19]:
# Setting up the deployment name
deployment_name = "gpt-4o-mini"

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
# This is the value of the endpoint for your Azure OpenAI resource
azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

# The API key for your Azure OpenAI resource.
api_key = os.environ['AZURE_OPENAI_KEY']

# Currently OPENAI API have the following versions available: https://learn.microsoft.com/azure/ai-services/openai/reference
api_version = "2024-07-01-preview"

client = openai.AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

## 1.0 Create the system message for ChatGPT

In [20]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


## 2.0 Define helper functions



In [39]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [59]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=max_response_tokens,
        temperature=0.7,
    )
    return response.choices[0].message.content

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [60]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [61]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[{"role": "user", "content": user_message}]

In [62]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 25


In [63]:
max_response_tokens = 100

response = send_message(messages, deployment_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
### The Impact of AI on the Future of Work: Navigating a New Era

As we step further into the 21st century, artificial intelligence (AI) continues to transform every facet of our lives, and the workplace is no exception. From automating repetitive tasks to enhancing decision-making processes, AI is reshaping the traditional work landscape in profound ways. In this blog post, we’ll explore the key areas where AI is making its mark and what the future of work might look like in an



## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [64]:
overall_max_tokens = 1096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [65]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name, max_response_tokens)

#Append assistance response 
messages.append({"role": "assistant", "content": response})
print_conversation(messages)

Token count: 153
[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
### The Impact of AI on the Future of Work: Navigating a New Era

As we step further into the 21st century, artificial intelligence (AI) continues to transform every facet of our lives, and the workplace is no exception. From automating repetitive tasks to enhancing decision-making processes, AI is reshaping the traditional work landscape in profound ways. In this blog post, we’ll explore the key areas where AI is making its mark and what the future of work might look like in an

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
### The Impact of AI on the Future of Work: A Guide for Tech Industry Leaders

As business leaders in the tech industry, you are at the forefront of one of the most transformative eras in human history. The integration of artificial intelligence (AI) into the workplace is not just a trend

### Option 2: Keep the conversation within a given number of turns

In [66]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [67]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name, max_response_tokens)

#Append assistant response
messages.append({"role": "assistant", "content": response})
print_conversation(messages)


[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
### The Impact of AI on the Future of Work: Navigating a New Era

As we step further into the 21st century, artificial intelligence (AI) continues to transform every facet of our lives, and the workplace is no exception. From automating repetitive tasks to enhancing decision-making processes, AI is reshaping the traditional work landscape in profound ways. In this blog post, we’ll explore the key areas where AI is making its mark and what the future of work might look like in an

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
### The Impact of AI on the Future of Work: A Guide for Tech Industry Leaders

As business leaders in the tech industry, you are at the forefront of one of the most transformative eras in human history. The integration of artificial intelligence (AI) into the workplace is not just a trend; it’s a seismic 

## Ejercicio 1: Modificar el Mensaje del Sistema
Enunciado:
Crea un mensaje del sistema que defina a ChatGPT como un "analista financiero experto". Imprime el mensaje del sistema resultante y utilízalo en un mensaje inicial del usuario que pregunte:
"¿Cuáles son las tendencias actuales en el mercado de acciones tecnológicas?".


In [68]:

# Mensaje del sistema
base_system_message = "You are a financial analyst expert."
system_message = f"{base_system_message.strip()}"
print(system_message)

# Crear lista de mensajes
user_message = "What are the current trends in the tech stock market?"
messages = [{"role": "user", "content": user_message}]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name, max_response_tokens)

messages.append({"role": "assistant", "content": response})
print_conversation(messages)


You are a financial analyst expert.
[USER]
What are the current trends in the tech stock market?

[ASSISTANT]
As of my last update in October 2021, I can't provide real-time data or current trends in the tech stock market. However, I can give you some general trends that were shaping the tech industry around that time, which may still be relevant:

1. **Cloud Computing Growth**: Companies focusing on cloud services, such as Amazon (AWS), Microsoft (Azure), and Google Cloud, were experiencing significant growth as businesses continued to migrate to cloud-based solutions.

2. **Artificial Intelligence and Machine



## Ejercicio 2: Contar Tokens en una Conversación Extendida
Enunciado:
Dado un conjunto inicial de mensajes, agrega tres mensajes adicionales:

"Explain the impact of inflation on stock prices."
"What are the top stocks to watch in 2025?"
"Summarize the latest market news in 50 words."
Calcula y muestra el número total de tokens tras añadir cada mensaje.


In [69]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a helpful assistant specialized in financial topics."},
    {"role": "user", "content": "What are the current trends in the tech stock market?"}
]

# Mensajes adicionales
additional_messages = [
    "Explain the impact of inflation on stock prices.",
    "What are the top stocks to watch in 2025?",
    "Summarize the latest market news in 50 words."
]

# Agregar mensajes y calcular tokens
for msg in additional_messages:
    messages.append({"role": "user", "content": msg})
    token_count = num_tokens_from_messages(messages)
    print(f"Token count after adding '{msg}': {token_count}")


Token count after adding 'Explain the impact of inflation on stock prices.': 48
Token count after adding 'What are the top stocks to watch in 2025?': 65
Token count after adding 'Summarize the latest market news in 50 words.': 82


## Ejercicio 3: Personalizar el Mensaje del Sistema Según el Usuario
Enunciado:
Escribe un programa que permita al usuario definir el rol de ChatGPT mediante una entrada de texto (input). Una vez definido, responde al mensaje:
"What are your key skills as an assistant?"
Usa el mensaje del sistema definido por el usuario.


In [70]:

# Entrada del usuario para definir el mensaje del sistema
user_defined_role = input("Define the role of the assistant: ")
system_message = f"You are {user_defined_role.strip()}."

# Crear lista de mensajes
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "What are your key skills as an assistant?"}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name, max_response_tokens)

messages.append({"role": "assistant", "content": response})

# Mostrar la conversación
print_conversation(messages)


[SYSTEM]
You are a helpful assistant specialized in fossils.

[USER]
What are your key skills as an assistant?

[ASSISTANT]
As an assistant specialized in fossils, my key skills include:

1. **Knowledge of Paleontology**: I have a solid understanding of fossil types, formation processes, and the history of life on Earth.

2. **Identification and Classification**: I can help identify and classify various fossils based on physical characteristics and geological context.

3. **Research Skills**: I can assist in finding and summarizing scientific literature related to fossils, paleoenvironments, and related fields.

4. **Educational Support**



## Ejercicio 4: Crear un Generador de Respuestas Creativo
Enunciado:
Diseña un programa que permita configurar la creatividad del modelo (temperatura) a través de una entrada del usuario. Una vez configurado, responde a la pregunta:
"Can you suggest an out-of-the-box idea for a startup?"



In [82]:

# Entrada del usuario para definir la creatividad
temperature = float(input("Set the creativity level (0.0 - 1.0): "))
if not (0.0 <= temperature <= 1.0):
    raise ValueError("Creativity level must be between 0.0 and 1.0.")

# Crear lista de mensajes
system_message = "You are a highly creative assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Can you suggest an out-of-the-box idea for a startup?"}
]

# Enviar mensaje con creatividad ajustada
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    max_tokens=1000,
    temperature=temperature
)

# Mostrar la respuesta
print(response.choices[0].message.content)

**Startup Idea: "MoodScape" - Personalized Ambient Environment Service**

**Concept:**
MoodScape is a subscription-based service that provides personalized ambient environments for homes, offices, and public spaces. Leveraging AI, smart devices, and IoT technology, MoodScape creates tailored atmospheres based on individual preferences, emotional states, or productivity needs.

**How It Works:**
1. **Personalized Assessment:** Users complete a quick survey or use an app feature to analyze their current mood, desired ambiance, or specific tasks they need to focus on (e.g., relaxation, creativity, or productivity).

2. **AI-Powered Environment Design:** The algorithm curates a unique combination of lighting, sound, temperature, and scent profiles. For example:
   - **Relaxation Mode:** Soft blue lighting, gentle ocean sounds, and lavender scent.
   - **Focus Mode:** Bright white lighting, subtle instrumental music, and eucalyptus scent.
   - **Creative Mode:** Warm yellow lighting, nature

## Ejercicio 5: Cambiar el Tono del Mensaje
Enunciado:
Configura el mensaje del sistema para que ChatGPT responda en un tono humorístico y escribe una respuesta al mensaje:
"Explain quantum computing in a funny way."


In [81]:

# Mensaje del sistema humorístico
system_message = "You are a humorous assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Explain quantum computing in a funny way."}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name, 1000)

#append assistant response
messages.append({"role": "assistant", "content": response})
print_conversation(messages)


[SYSTEM]
You are a humorous assistant.

[USER]
Explain quantum computing in a funny way.

[ASSISTANT]
Alright, imagine you have a really special cat, let’s call him Schrödinger's Cat. Now, Schrödinger's Cat isn’t just any ordinary cat; he’s both alive and dead at the same time! Why? Because he’s stuck in a quantum box, and he refuses to choose a side – it’s a classic case of feline indecision.

Now, quantum computing is like having a whole army of Schrödinger's Cats, but instead of just lounging around in boxes, they’re computing things at lightning speed! In the classical world, computers are like really diligent tortoises that can only handle one task at a time. They’re great at following instructions, but if you ask them to multitask, they’ll just stare at you as if you’ve asked them to solve a Rubik’s Cube while riding a unicycle.

But quantum computers? They’re like hyperactive rabbits that can simultaneously hop in multiple directions! They use qubits, which are like the magical 

## Ejercicio 6: Resumir Mensajes Previos
Enunciado:
Dado un historial de conversación de 5 mensajes, crea una función que resuma el contenido de los mensajes del usuario en una sola oración.



In [83]:

# Historial de mensajes
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me about AI."},
    {"role": "user", "content": "How does machine learning work?"},
    {"role": "user", "content": "Can you explain neural networks?"},
    {"role": "user", "content": "What are the applications of deep learning?"}
]

# Resumir mensajes del usuario
def summarize_user_messages(messages):
    user_messages = [msg['content'] for msg in messages if msg['role'] == 'user']
    summary_prompt = "Summarize the following messages into one sentence: " + " ".join(user_messages)
    summary = send_message([{"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user", "content": summary_prompt}], deployment_name)
    return summary

# Obtener y mostrar el resumen
summary = summarize_user_messages(messages)
print(f"User messages summary: {summary}")


User messages summary: AI involves the use of machine learning, which employs algorithms to learn from data, including neural networks that simulate human brain functions, and has various applications in fields like image recognition, natural language processing, and autonomous systems.


## Ejercicio 7: Configuración de Temperatura y Tono
Enunciado:
Escribe un programa que cambie la configuración de temperatura a 0.9 para obtener respuestas más creativas y responde al mensaje:
"Suggest a unique title for a blog about AI in education."



In [85]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a creative assistant."},
    {"role": "user", "content": "Suggest a unique title for a blog about AI in education."}
]

# Ajustar temperatura y enviar mensaje
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    max_tokens=1000,
    temperature=0.9
)

# Mostrar la respuesta
print(response.choices[0].message.content)


"EduTech Evolution: Navigating the AI Frontier in Learning"
